In [2]:
#imports 
from bs4 import BeautifulSoup
import requests
import pprint
import pandas as pd



Initial Editing of XML files

contributors/collaborators: Peter Nadel

In [ ]:
#initial XML
!wget https://raw.githubusercontent.com/gregorycrane/Wolf1807/master/wolf.aw.xml
wolf= open('wolf.aw.xml')
wolf_soup = BeautifulSoup(wolf,'lxml')
print(wolf_soup) 

In [ ]:
#using beautiful soup to find all the p or paragrapg tags 
paragraphs = wolf_soup.find_all('p')[0].find_all('p')
paragraphs = paragraphs[3:] ## cleaning out other tags

#numbering paragraphs using the index 
for index, ptag in enumerate(paragraphs): 
  ptag['id'] = f'para{index}'

#using nltk to split the paragraphs into sentences 
#using the same beautiful soup process except creating new s tags for sentences 
#using the index of the sentence to number them
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')
for p_index, paragraph in enumerate(paragraphs):
  sentences = sent_tokenize(paragraph.get_text())
  paragraph.clear()
  for index, sentence in enumerate(sentences):
    sent_tag = wolf_soup.new_tag('s', id= f'para{p_index}s{index}') 
    sent_tag.string = sentence 
    paragraph.append(sent_tag) 

paragraphs

Paragraphs output became the initial working xml file.
It required some manual edits....
Things I did manually:
- Some of the sentence tokenization split up initials into individual sentences like A.b. Milne was split into three sentences
- I had to manually re add in the notes tags 

Final XML can be found here : [https://github.com/jdeen33/Altertumswissenschaft-/blob/main/wolf_attributes.xml]

**Generating ConLLu Files From Each Sentence**

contributors/collaborators: Peter Nadel

In [ ]:
#imports
from google.colab import files

uploaded = files.upload()

!pip install spacy
import spacy
!python -m spacy download de_core_news_md 
nlp = spacy.load("de_core_news_md")
!pip install spacy_conll
from spacy_conll import init_parser
# include spacy-stanza and spacy-udpipe
!pip install spacy_conll[parsers]
# include pandas
!pip install spacy_conll[pd]
!pip install spacy_conll[all]
from spacy_conll import init_parser

nlp = init_parser("de",
                  "stanza",
                  parser_opts={"use_gpu": True, "verbose": False},
                  include_headers=True)
config = {"ext_names": {"conll_pd": "pandas"},
          "conversion_maps": {"deprel": {"nsubj": "subj"}}}

In [ ]:
#making a dictionary w/ tag id as key
#get_text as value 
#for every sentence in Wolf 
wolf= open('wolf_attributes.xml')
wolf_soup = BeautifulSoup(wolf,'lxml')

sents = wolf_soup.find_all('s')
s_dict= {}
for item in sents:
  s_dict[item['id']] = item.get_text()

s_dict

In [ ]:
#convert dictionary to pandas dataframe
import pandas as pd
wolf_s_tags = pd.DataFrame.from_dict(s_dict, orient='index').rename(columns={0:'StringText'},)
wolf_s_tags.reset_index().rename(columns={'index':'p_id'})
wolf_s_tags
#making directory to store pandas files 
!mkdir pandas_csv

In [ ]:
#I used a for loop w/ range to do this, which is very inefficient 
# but it kept throwing errors when a sentence tag was empty and interrupting the entire process
#i think if I rewrote it with a try/except loop it would work
#but anyways this uses spacy to convert the tokenizer output into a pandas dataframe,which is then converted into csv 
for i in range (901,969):
  x =wolf_s_tags['StringText'][i]
  doc = nlp(x)
  s = doc._.conll_pd
  s.to_csv(f"pandas_csv1/{wolf_s_tags.index[i]}.csv")

In [ ]:
#used python 'glob' tool to grab all the conllu csv files from the directory and make them into one huge dataframe 
import glob
import pandas as pd
sent_files = sorted(glob.glob('/content/pandas_csv1/para*.csv'))
sent_files
second_half = pd.concat((pd.read_csv(file) for file in sent_files))

This file can be found here :[https://github.com/jdeen33/Altertumswissenschaft-/blob/main/wolf_sents_entire]

it needs to be updated w/ the new xml file, which I will do soon

**Scraping Google Ngrams**

sources used: https://www.geeksforgeeks.org/scrape-google-ngram-viewer-using-python/

In [ ]:
#imports 
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns


In [ ]:
#https://www.geeksforgeeks.org/scrape-google-ngram-viewer-using-python/
import urllib
import requests
import json
def runQuery(query, start_year, 
             end_year, corpus,
             smoothing):
  
    # converting a regular string to 
    # the standard URL format
    # eg: "geeks for,geeks" will
    # convert to "geeks%20for%2Cgeeks"
    out_dict = {}
    query = urllib.parse.quote(query)
  
    # creating the URL
    url = "https://books.google.com/ngrams/json?content=" + query +"&year_start=" + str(start_year) + "&year_end=" + str(end_year) + "&corpus=" + str(corpus) + "&smoothing=" + str(smoothing) + ""
  
    # requesting data from the above url
    response1 = requests.get(url)
    
    # extracting the json data from the response we got
    output = response1.json()
    #print(output)
    for num in range(len(output)):
      if len(output)== 0:
        continue
      else:
        #return(list(output[num]['timeseries']))
        out_dict[output[num]['ngram']] = list(output[num]['timeseries'])
      return out_dict
 
    
      

     # y = (output[num]['timeseries'])
     # array = (np.array(y))
     # return array
      

In [ ]:
# I forget how I made this list 
# I think I just took the words from the lemmas column of my giant connlu dataframe
#filtering punctuation etc 

lemmas = []
for item in lems:
  if item in lemmas:
    continue
  elif item == '.':
    continue
  elif item == ',':
    continue
  elif item == ';':
    continue
  else:
    lemmas.append(item)
print(lemmas)

In [ ]:
#making a directory for the csv files of ngrams data
!mkdir ngrams_data

In [ ]:
for item in lemmas:
  try:
    x = runQuery(item, start_year=1700, end_year=1900, corpus=31, smoothing=0)
  except Exception as e:
    pass
    #gets only numbers from google ngrams output
    ngdata =pd.DataFrame.from_dict(x, orient='columns') 
      #not the most efficient but- makes an array with the word to attach it to data
    dayu= np.repeat(item,201)
    gram= np.reshape(dayu,(201,1))
    ls = pd.DataFrame(gram)
    final = pd.concat([ls,ngdata], axis= 1)
      #this gives one column of the word, and one column of the ngrams
      #altogether, this dataframe will be whats on the y axis of heatmap
    final.rename(columns = {0:'Ngram'}, inplace = True)
      #adds the year range which is in the same order as data scraped from google ngrams
    list1 = list(np.arange(1700,1900+1))
    yrs =pd.DataFrame(list1)
    yrs.rename(columns={0:'Years'}, inplace=True )
      #adds 2 dataframes together
    horizontal_stack = pd.concat([final, yrs], axis=1)
    horizontal_stack.rename(columns={ horizontal_stack.columns[1]: "NGdata" }, inplace = True)
      #write to csv
    horizontal_stack.to_csv(f"/content/ngrams_data/{item}.csv",)

ngrams_data file can be found here : https://github.com/jdeen33/Altertumswissenschaft-/blob/main/wolfngrams.csv 

In [ ]:
#then used bins and pivot tables to organize data. Data organized in 10 year increments for each word, from the years 1700-1900
#splitting data for years into intervals of 10
bins = [1700,1710,1720,1730,1740,1750,1760,1770,1780,1790,1800,1810,1820,1830,1840,1850,1860,1870,1880,1890,1900]
wolf['years'] = pd.cut(wolf['Years'], bins)
wolf

binned csv can be found here:
https://github.com/jdeen33/Altertumswissenschaft-/blob/main/pivotwolfngrams.csv

In [ ]:
#making a heatmap 
from matplotlib.pyplot import figure
df_wide = wolf.pivot_table( index= 'Ngram' ,columns='years', values='NGdata')
l = df_wide.sample(n=100) #shows a random sample of 100 words 
figure(figsize=(10, 8), dpi=100)
# plot it
sns.heatmap(l)

heatmap example can be found here:
https://github.com/jdeen33/Altertumswissenschaft-/blob/main/ngrams_heatmap.png 

Using DeepL API to automate an english translation 

In [ ]:
#imports 
import toolz
from google.colab import files
uploaded = files.upload()
#used new XML (wolf_attributes- link to github file above)
!pip install --upgrade deepl
import deepl

In [ ]:
wolf= open('wolf_attributes.xml')
wolf_soup = BeautifulSoup(wolf,'lxml')


sents = wolf_soup.find_all('s')
s_dict= {}
for item in sents:
  s_dict[item['id']] = item.get_text()


In [ ]:
#data cleaning 
import re
pattern = "\n\t\t\t"
pattern2 = "\s\\n*"
def cleanxml(x):
  #return re.sub(pattern, " ", x)
  return re.sub(pattern2, " ", x)

#removing all the line and tab breaks
def map2dictionary(x,d):
  cleandict = toolz.valmap(cleanxml,s_dict)
  return cleandict

In [ ]:
wolf_dict = map2dictionary(cleanxml,s_dict)

In [ ]:
#clean data to run through api 
wolf_dict

In [ ]:
#deepL is unfortunately behind a paywall but I used a free subscription 
auth_key = "a4713cc6-2a17-9891-4c5a-8ef473dd80dd"
translator = deepl.Translator(auth_key)

In [ ]:
#using dict comprehension to translate each sentence 
def translate_dict(dict_):
  wolf_eng = {}
  for key in dict_:
    if len(dict_[key]) > 0 :
      result= translator.translate_text(dict_[key],target_lang="EN-US")
      wolf_eng[key] = result.text
  return wolf_eng
#concat into dataframe then download as csv file 
wolf_pd = pd.DataFrame.from_dict(translate_dict(wolf_dict), orient='index')
wolf_pd.to_csv('wolf_automated_eng', encoding = 'utf-8-sig') 
files.download('wolf_automated_eng')


Link to this csv can be found here:
https://github.com/jdeen33/Altertumswissenschaft-/blob/main/wolf_automated_eng.txt 